In [2]:
import numpy as np
import matplotlib.pyplot as plt

class Love_wave():
    # parameter in SI unit
    def __init__(self, n, structure):
        
        # shallow structure
        self.H  = structure[0]                           # thickness(m) 
        self.b1 = structure[1]                           # velocity (m/s)
        self.d1 = structure[2]                           # density  (kg*m^-3)
        self.u1 = self.b1**2*self.d1                     # shear modulus
        
        # deep background
        self.b2 = structure[3]                           # velocity (m/s)
        self.d2 = structure[4]                           # density  (kg*m^-3)
        self.u2 = self.b2**2*self.d2                     # shear modulus
        
        # model parameter
        self.n    = n                                    # nth overtone
        self.intv = 1/(10+abs(0.3-n)**2)                 # calculating step
        self.num  = int((self.b2-self.b1)/self.intv+1)   # calculating times
        
        # modeling
        self.c = np.linspace(self.b1, self.b2, self.num) # phase velocity
    
    def overtone(self, w, plot=True):
        dv1 = np.sqrt(self.b1**(-2) - self.c**(-2))
        dv1[dv1==0.0] = 10**-16
        dv2 = np.sqrt(self.c**(-2)  - self.b2**(-2))
        w_y = dv1*self.H*w
        RHS = (dv2*self.u2)/(dv1*self.u1)
        LHS = np.tan(w_y)
        
        # numerical solution
        fi = []
        x  = []
        y  = []
        for i in range(1,self.c.shape[0]):
            aa = (RHS[i-1]-LHS[i-1])*(RHS[i]-LHS[i])
            if aa <= 0:
                fi.append(i)
        for i in range(len(fi)//2):
            j = np.where(w_y == w_y[fi[2*i]])[0][0]
            x.append(w_y[j])
            y.append(RHS[j])
            print(f'{i} overtune: {x[i].round(3)}, {y[i].round(3)}')
        if plot==True:            
            LHS[:-1][np.diff(LHS) < 0] = np.nan
            plt.plot(w_y, LHS, 'b', label='LHS')        
            plt.plot(w_y, RHS, 'r', label='RHS')
            plt.scatter(x, y, c='g')
            for i in range(len(x)):
                plt.text(x[i]+0.1, y[i]+0.1, f'n={i}')
            plt.hlines(0.0, min(w_y), max(w_y), 'k')
            plt.xlim(min(w_y), max(w_y))
            plt.ylim(-5.0, 40.0)
            plt.title(f'ω = {w}')
            plt.legend(loc='upper right')
            plt.show()
        
    def frequency(self):
        dv1 = np.sqrt(self.b1**(-2) - self.c**(-2))
        dv1[dv1==0.0] = 10**-16
        dv2 = np.sqrt(self.c**(-2)  - self.b2**(-2))
        # analytic solution
        w = np.arctan( (self.u2*dv2)/(self.u1*dv1) )
        w = (w + self.n*np.pi)/(self.H*dv1)
        return w, self.c

    def velocity(self, w): # U(c)
        k = w/self.c
        n = self.c.shape[0]
        dc_dk = np.zeros([n])
        for i in range(1, n):
            dc_dk[i] = (self.c[i]-self.c[i-1])/(k[i]-k[i-1])
        U = self.c + k*dc_dk
        U[0] = self.c[0]
        return U, self.c


#=====================#
#    新增的函式段落    #
#=====================#
def get_reference_velocity(structure, omega):
    """
    給定結構與目標角頻率陣列，
    回傳對應參考相位速度陣列 C_ref。
    """
    wave = Love_wave(0, structure)
    w_model, c_model = wave.frequency()
    
    C_ref = []
    for w in omega:
        # 找出距離 w 最近的頻率索引
        idx = np.argmin(np.abs(w_model - w))
        C_ref.append(c_model[idx])
    return np.array(C_ref)


def inversion(structure, C_ref, C_obs, delta, omega):
    """
    structure: 結構參數 (含 V1, V2)
    C_ref    : 參考相位速度 (來自 get_reference_velocity)
    C_obs    : 觀測到的相位速度
    delta    : [delV1, delV2] 用來做有限差分
    omega    : 角頻率陣列
    """
    # 參考模型
    m_ref = np.array([structure[1], structure[3]])  # (V1, V2)
    delV1, delV2 = delta

    # 資料向量 d = (C_obs - C_ref)
    d = np.array(C_obs) - np.array(C_ref)
    
    # G矩陣 (4 x 2)
    G = np.zeros([len(omega), 2])
    
    #-----------------------------#
    #   改動 1：計算 ∂C/∂(V1)    #
    #-----------------------------#
    # 先把結構調回參考值
    structure[1] = m_ref[0]
    structure[3] = m_ref[1]
    # 只 perturb V1
    structure[1] += delV1
    
    wave1 = Love_wave(0, structure)
    w_m1, c1 = wave1.frequency()
    
    for i in range(len(omega)):
        w = omega[i]
        idx = np.argmin(np.abs(w_m1 - w))
        # 有限差分
        G[i, 0] = (c1[idx] - C_ref[i]) / delV1
    
    #-----------------------------#
    #   改動 2：計算 ∂C/∂(V2)    #
    #-----------------------------#
    # 結構再歸回參考值
    structure[1] = m_ref[0]
    structure[3] = m_ref[1]
    # 只 perturb V2
    structure[3] += delV2
    
    wave2 = Love_wave(0, structure)
    w_m2, c2 = wave2.frequency()
    
    for i in range(len(omega)):
        w = omega[i]
        idx = np.argmin(np.abs(w_m2 - w))
        # 有限差分
        G[i, 1] = (c2[idx] - C_ref[i]) / delV2

    #------------------------------#
    #   改動 3：做最小平方反演     #
    #------------------------------#
    # d = G * delta_m  ==>  delta_m = (G^T G)^{-1} G^T d
    # 因為 G.shape=(4,2) 非方陣，故直接用 lstsq() 做
    delta_m, residuals, rank, s = np.linalg.lstsq(G, d, rcond=None)
    m_new = m_ref + delta_m  # 新的 (V1, V2)
    
    return m_new


#------------------#
#   主程式測試段   #
#------------------#
if __name__ == "__main__":
    # 頻率 & 角頻率
    f = np.array([0.08, 0.15, 0.18, 0.22])
    omega = 2 * np.pi * f
    
    # 結構參數
    structure = [
        10000.0,  # 0 thickness(m)
        3000.0,   # 1 velocity (m/s)
        2800.0,   # 2 density  (kg*m^-3)
        5000.0,   # 3 velocity (m/s)
        3300.0    # 4 density  (kg*m^-3)
    ]

    # 觀測到的相位速度
    C_obs = [4130, 3529, 3427, 3346]

    #------------------------------------------------------#
    #   改動 4：計算參考相位速度 C_ref (原程式缺少的部分)    #
    #------------------------------------------------------#
    C_ref = get_reference_velocity(structure, omega)
    print("Reference phase velocity (C_ref) =", C_ref.round(2))

    delta = [1.0, 1.0]

    print("== Theoretical dispersion ==")
    for i in range(len(f)):
        print(f"freq = {f[i]:.3f} Hz,  c_theory = {C_ref[i]:.2f} m/s")

    # 進行反演
    m = inversion(structure, C_ref, C_obs, delta, omega)

    print('\n>> Inversion result:')
    print('β1 = ', round(m[0], 2), 'm/s')
    print('β2 = ', round(m[1], 2), 'm/s')

Reference phase velocity (C_ref) = [4055.3  3335.98 3234.99 3158.57]
== Theoretical dispersion ==
freq = 0.080 Hz,  c_theory = 4055.30 m/s
freq = 0.150 Hz,  c_theory = 3335.98 m/s
freq = 0.180 Hz,  c_theory = 3234.99 m/s
freq = 0.220 Hz,  c_theory = 3158.57 m/s

>> Inversion result:
β1 =  3158.05 m/s
β2 =  4734.41 m/s
